In [1]:
import pandas as pd
import numpy as np

# Carga de datos

In [2]:
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored= pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

# Preprocesado de los datos

### Eliminación de Features non scored con pocos casos positivos

In [3]:
y_nonscored = train_targets_nonscored.drop('sig_id', axis = 1, inplace = False)

In [4]:
ones = []
bound = 25 #@param {type:"slider", min:1, max:50, step:1}
for col in y_nonscored.columns:
  if(y_nonscored[col].sum() > bound):
    ones.append(col)

In [5]:
y_nonscored = y_nonscored[ones]
len(ones)

42

In [6]:
#Model Features
X =  train_features.drop('sig_id', axis = 1, inplace = False)
X = pd.get_dummies(X, columns = ['cp_type', 'cp_time', 'cp_dose'])

#Scored target
y_scored = train_targets_scored.drop('sig_id', axis = 1, inplace = False)
#Non scored target
#y_nonscored = train_targets_nonscored.drop('sig_id', axis = 1, inplace = False)
#Merged target
y = pd.concat([y_scored,y_nonscored],  axis = 1, sort = False)

#Targets test
y_test = sample_submission.copy()
for col in y_test.loc[:, y_test.columns != 'sig_id'].columns:
    y_test[col].values[:] = 0
    
#Test features
sig_id_test = test_features['sig_id']
X_test = test_features.drop('sig_id', axis = 1, inplace = False)
X_test = pd.get_dummies(X_test, columns = ['cp_type', 'cp_time', 'cp_dose'])

# Modelo

## Estructura

In [7]:
import tensorflow as tf
import tensorflow_addons as tfa
import random
import os
from tensorflow.keras.callbacks import ReduceLROnPlateau

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [8]:
def model():
  model = tf.keras.Sequential([
      #Input layer
      tf.keras.layers.Input(879),
      #Dense layer 1
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.2),
      tfa.layers.WeightNormalization(tf.keras.layers.Dense(2096*1.2, activation = "relu")),
      #Dense layer 2
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.5),
      tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024*1.2, activation = "relu")),
      #Output layer
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.4),
      tfa.layers.WeightNormalization(tf.keras.layers.Dense(248, activation = "sigmoid")),
  ])

  model.compile(optimizer = tfa.optimizers.Lookahead(tf.keras.optimizers.Adam()),
                loss = 'binary_crossentropy',
                metrics = ['accuracy', 'AUC'])
  
  return model

## Entrenamiento

In [9]:
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [10]:
NFOLD = 16
kf = KFold(n_splits = NFOLD)

BATCH_SIZE = 32
EPOCHS = 216

#Valores en forma de array para poder enchufarlos a la red
X_vals = X.values
y_vals = y.values
val_pred = np.zeros((train_features.shape[0], 248)) #Grid vacia para la predicción del validation split
test_pred = np.zeros((test_features.shape[0], 248)) #Grid vacía par ala predicción del test

#Kfold para train set y validation set
cnt = 1
for tr_idx, val_idx in tqdm(kf.split(X_vals)):

  #Reduccion del learning rate a medida que nuestro modelo se estanca
  reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', 
                                     factor=0.1, 
                                     patience=3, 
                                     verbose=1,
                                     min_delta=1e-4, 
                                     mode='min')

  print(f'FOLD {cnt}')
  cnt+=1

  net = model()
  net.fit(X_vals[tr_idx], y_vals[tr_idx], 
          batch_size = BATCH_SIZE,
          epochs = EPOCHS,
          validation_data = (X_vals[val_idx], y_vals[val_idx]),
          verbose = 0,
          callbacks = [reduce_lr_loss])
    
  #Impresión de los resultados
  print("TRAIN : \n", net.evaluate(X_vals[tr_idx],y_vals[tr_idx],
                                  verbose = 0,
                                  batch_size = BATCH_SIZE))

  print("VAL  : \n", net.evaluate(X_vals[val_idx], y_vals[val_idx],
                                  verbose = 0,
                                  batch_size = BATCH_SIZE))

  #Predicción de los valores de validación
  val_pred[val_idx] = net.predict(X_vals[val_idx],
                                  batch_size = BATCH_SIZE,
                                  verbose = 0)

  print("Validation predicted")

  #Predicción final del test
  test_pred += net.predict(X_test,
                          batch_size = BATCH_SIZE,
                          verbose = 0)/ NFOLD

  print("Test predicted")

FOLD 1

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.

Epoch 00033: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.

Epoch 00036: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-11.

Epoch 00039: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-12.

Epoch 00042: ReduceLROnPlateau reducing learning rate to 1.0000001044244145e-13.

Epoch 00045: ReduceLROnPlateau reducing learning rate to 1.0000001179769417e-14.

Epoch 00048: ReduceLROnPlateau reducing learning rate to 1.0000001518582595e-15.

Epoch 00051: 

In [11]:
columns = y_scored.columns # Cogemos el nombre de las columnas

final_sub  = pd.DataFrame(data = test_pred[:,:206], columns = columns)
final_sub.insert(0, column = 'sig_id', value = y_test['sig_id'])

# Submission

In [12]:
final_sub.to_csv('submission.csv', index = False)

In [13]:
y.shape

(23814, 248)

In [14]:
val_pred.shape

(23814, 248)

In [15]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

In [16]:
log_loss_metric(y_scored,val_pred[:,:206])

0.014936984069045109

In [17]:
v4 = 0.016096644893067273
v5 = 0.0158627008875029
v6 = 0.01577834332535018
v7 = 0.015266542185011646
v8 = 0.015230647531349884

# Versiones guardadas
- Fallidas:
    - Versión 2 con 4096 neuronas empeora el resultado a 0.01632391606043724


- Version 4: submission 4 : 0.016096644893067273 (NN simple con una deep layer)
    - Fallos: Faltaba la última capa de BatchNormaliztion
- Version 5: 0.0158627008875029
    - Mejora del fallo de la versión 4
- Versión 6 : 0.01577834332535018
     - Cambio de la capa de DropOut a 0.4
- Versión 7: 0.015266542185011646
    - Nueva capa densa
    - Uso de un nuevo optimizador
    - Uso de normalización de pesos en cada capa
    - Añadidas más capas de dropout
- Version 8: 0.015212403062810905
    - Reajuste manual de los pesos
    